In [1]:
import pandas as pd
import numpy as np
import time
import psutil
import subprocess
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input

In [2]:
data= pd.read_csv(r"C:\Users\hsynt\Downloads\Selcuk Abi\Selcuk Abi\VeriSeti2.csv")

In [3]:
# GPU Kullanımını Ölçmek İçin Fonksiyon
def get_gpu_usage():
    try:
        result = subprocess.check_output(["nvidia-smi", "--query-gpu=utilization.gpu", "--format=csv,noheader,nounits"])
        gpu_usage = float(result.decode("utf-8").strip())
    except:
        gpu_usage = 0.0  # GPU yoksa veya erişilemezse 0 olarak döndür
    return gpu_usage

# CPU ve Bellek Kullanımını Ölçmek İçin Fonksiyon
def get_cpu_memory_usage():
    cpu_usage = psutil.cpu_percent(interval=1)
    memory_usage = psutil.virtual_memory().used / (1024 ** 3)  # GB cinsinden
    return cpu_usage, memory_usage

# Özellik ve Hedef Ayrımı
X = data.drop(columns=['Target_10', 'adjclose'])
y = data['Target_10']

# Zaman Serisi Formatına Getirme
def create_sequences(data, target, seq_length):
    sequences, targets = [], []
    for i in range(len(data) - seq_length):
        sequences.append(data[i:i + seq_length].flatten())  # MLP için düzleştirilmiş giriş
        targets.append(target[i + seq_length])
    return np.array(sequences), np.array(targets)

SEQ_LENGTH = 10
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_seq, y_seq = create_sequences(X_scaled, y.values, SEQ_LENGTH)

# PCA ve LDA Veri Hazırlığı
pca = PCA(n_components=10)
X_pca = pca.fit_transform(X_seq)

lda = LDA(n_components=1)
X_lda = lda.fit_transform(X_seq, y_seq)

# MLP Modeli Tanımlama
def build_mlp_model(input_shape):
    model = Sequential([
        Input(shape=(input_shape,)),
        Dense(128, activation='relu'),
        Dropout(0.3),
        Dense(64, activation='relu'),
        Dropout(0.2),
        Dense(32, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Performans Hesaplama
results = []

for reduction, X_data in zip(
    ['No PCA/LDA', 'PCA', 'LDA'],
    [X_seq, X_pca, X_lda]
):
    # Modeli Oluştur
    mlp_model = build_mlp_model(X_data.shape[1])
    
    # Modeli Eğit ve Kullanım Ölçümleri
    start_time = time.time()
    mlp_model.fit(X_data, y_seq, epochs=3, batch_size=32, verbose=1)
    elapsed_time = time.time() - start_time

    # Kullanım Ölçümleri
    gpu_usage = get_gpu_usage()
    cpu_usage, memory_usage = get_cpu_memory_usage()

    results.append({
        'Reduction': reduction,
        'Model': 'MLP',
        'CPU Usage (%)': cpu_usage,
        'GPU Usage (%)': gpu_usage,
        'Memory Usage (GB)': memory_usage
    })

# Performans Sonuçlarını Görselleştirme
results_df = pd.DataFrame(results)

# CPU, GPU ve Bellek Kullanımı Tablosu
print("CPU, GPU ve Bellek Kullanımı Tablosu:")
print(results_df)

# Sonuçları Kaydetme
results_df.to_csv("mlp_resource_usage.csv", index=False)
print("\nSonuçlar 'mlp_resource_usage.csv' dosyasına kaydedildi.")

Epoch 1/3
1562/1562 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.8361 - loss: 0.4000
Epoch 2/3
1562/1562 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.8686 - loss: 0.3151
Epoch 3/3
1562/1562 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.8735 - loss: 0.2983
Epoch 1/3
1562/1562 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.7617 - loss: 0.5227
Epoch 2/3
1562/1562 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.7952 - loss: 0.4729
Epoch 3/3
1562/1562 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.7985 - loss: 0.4686
Epoch 1/3
1562/1562 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.8590 - loss: 0.3436
Epoch 2/3
1562/1562 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.8666 - loss: 0.3141
Epoch 3/3
1562/1562 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.8679 - loss: 0.3096
CPU, GPU ve Bellek Kullanımı Tablosu:
    Reduction Model  CPU Usage (%)  GPU Usage (%)  Memory Usage (GB)
0  No PCA/LDA   MLP            1.1            0.0          15.616669
1         PCA   MLP            0.1       